# **Helper Functions**

***

##1. The following is a working function to list th entire index from S3 directly utlizing boto3.

In [0]:
import boto3
import botocore
import pandas as pd

def list_crawls():
    # Set bucket name and prefix for crawl data
    bucket_name = "commoncrawl"
    prefix = "crawl-data/CC-MAIN-2025-33/"
    
    # Create an S3 client with credentials and region
    s3 = boto3.client(
        "s3",
        region_name='us-east-1',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key
    )
    
    # List objects in the specified prefix within the bucket
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    paginator = s3.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):
        object_keys.extend(page.get("Contents", []))

    # Extract object keys from the response
    object_keys = [
        content["Key"]
        for content in response.get("Contents", [])
    ]
    
    # Display the object keys as a DataFrame if any objects are found
    if object_keys:
        df_list_crawls = pd.DataFrame({"object_key": object_keys})
    else:
        # Print message if no objects are found
        print("No objects found.")
    return df_list_crawls
